# Modelling toolbox: ODE

*21st September 2021* - **Credits:** Luca Ciandrini (luca.ciandrini@umontpellier.fr)

---
Last week we introduced Ordinary Differential Equations (ODE):
$$
\frac{d}{dt} \mathbf{x}(t) = \mathbf{f}(\mathbf{x}(t)) \,
$$
and we guessed the solution of a few simple examples. Then we tackled a problem that it cannot be studied with deterministic approaches because of its intrinsic stochastic properties: the partition problem. We looked at the *fluctuations* in the partition of $N$ molecules from a mother in the two daughter cells, and we related that result to experimental outcomes.

After reading [Wil] D.J. Wilkinson, *Stochastic modelling for quantitative description of heterogeneous biological systems*, Nature Reviews (2009) [doi:10.1038/nrg2509](doi:10.1038/nrg2509):
- Discuss deteministic *vs* stochastic model;
- Name a few approaches and explain them;
---

## Step 1. Guess an analytical solution

Let's start with the example from [Wil]:
$$
\frac{d X}{dt} = \alpha - \mu X \,.
$$

How can you solve it analytically?
...
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

You should find 
$$
X(t) = \frac{\alpha}{\mu}(1-e^{-\mu t}) \,.
$$
What can you say about that?
...
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
Let's plot it now.

In [ ]:
import matplotlib.pyplot as plt ## matplotlib is a module for plotting in python
import numpy as np

alpha = 1
mu = 0.1 ## What units are these constants?

t = np.arange(0,100,.1) ## In array you have the indipendent variable t
print(t)

In [ ]:
X = (alpha/mu)*(1.-np.exp(-mu*t))  ## In this array you are going to put your values of X computed with the solution you found.
plt.plot(t,X)

const = alpha/mu 
plt.plot([0.,t[-1]], [const, const], '--')

plt.ylim(0.,20.)
plt.xlim(0.,t[-1])
plt.xlabel('t', fontsize = 14)
plt.ylabel('X', fontsize = 14)
plt.title('Analytical solution')
plt.show()

- Discuss the *steady state*.

## Step 2. Numerical solution
Numerical solvers solve differential equations in an iterative fashion, starting from the initial conditions. Numerically, the derivative can be approximated using *finite differences*.

Let $\Delta t$ be a very small step, then
$$
\mathbf{f}(\mathbf{x}(t)) \simeq \cfrac{\mathbf{x}(t+\Delta t) - \mathbf{x}(t)}{\Delta t}
$$
so $\mathbf{x}(t+\Delta t) \simeq \mathbf{x}(t) + \mathbf{f}(\mathbf{x}(t)) \Delta t$, at the foundations of the Euler method 
$$
\mathbf{x}_{i+1} = \mathbf{x}_i + \mathbf{f}(\mathbf{x}_i)\Delta t.
$$

Let's try to develop this simple integration method.

In [ ]:
#function that takes a(alpha), b (mu) and z (x) as inputs
def f(a,b,z):
    return a - b*z

In [ ]:
delta = 0.01 # definition of Delta t
t_max = 100  # Maximal length of integration - final time

x_num = []   # We will build here the numerical solution
t_num = []   # and here the time steps

# Set the initial conditions:
time = 0.
t_num.append(0.)
x_num.append(0.)

while (time < t_max):
    time += delta
    t_num.append(time)
    x_prev = x_num[-1]
    x_next = x_prev + f(alpha, mu, x_prev)*delta
    x_num.append(x_next)

In [ ]:
plt.plot(t_num,x_num, label='numerical')

plt.plot([0.,t[-1]], [const, const], '--')

plt.plot(t,X, 'gray', label='analytical') 


plt.ylim(0.,20.)
plt.xlim(0.,t[-1])
plt.xlabel('t', fontsize = 14)
plt.ylabel('X', fontsize = 14)
plt.title('Numerical solution')
plt.legend()
plt.show()

---
- What could be the problems of this method?
- Try to change the $\Delta t$

You should use already developed packages for numerical integrations. For instance:
[Scipy integrate](https://docs.scipy.org/doc/scipy/reference/reference/integrate.html)
```
scipy.integrate.solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, dense_output=False, events=None, vectorized=False, args=None, **options)
```

In [ ]:
from scipy.integrate import solve_ivp

def func(t, x, a, b): return a - b*x

sol = solve_ivp(func, [0, 100], [0.], args=[alpha, mu])
#sol = solve_ivp(func, [0, 100], [0.], args=[alpha, mu], t_eval=t)
#sol = solve_ivp(func, [0, 100], [0.], args=[alpha, mu], method='LSODA')


In [ ]:
sol

In [ ]:
sol.t

In [ ]:
sol.y

In [ ]:
plt.plot(sol.t, sol.y[0], label='solve_ivp')

plt.plot([0.,t[-1]], [const, const], '--')

plt.plot(t,X, 'gray', label='analytical') 


plt.ylim(0.,12.)
plt.xlim(0.,t[-1])
plt.xlabel('t', fontsize = 14)
plt.ylabel('X', fontsize = 14)
plt.title('Numerical solution from solve_ivp')
plt.legend()
plt.show()

# Enzyme reaction and Michaelis-Menten kinetics
Now let's give a look at the system of ODEs you would write to study Enzyme Reaction kinetics, and the Michaelis-Menten approximation.

Let's first define the set of ODEs for
$$
S + E \rightleftharpoons C \rightarrow P + E 
$$
with $k_1$ adn $k_2$ the rate constants (forward and backward respectively) for the first step, $k_3$ for the second. Concentrations are indicated with the lower case letters $s,e,c,p$.

In [ ]:
def enzymatic_reaction(t, y, k1, k2, k3):
    s, e, c, p = y
    return [-k1*s*e + k2*c, -k1*s*e + k2*c + k3*c, k1*s*e - k2*c - k3*c, k3*c]

In [ ]:
k1 = 1.
k2 = 0.1
k3 = 1.

e_tot = 1.
s0 = 1.

sol = solve_ivp(enzymatic_reaction, [0, 10], [s0, e_tot, 0., 0.], args=[k1, k2, k3], method='LSODA')


In [ ]:
sol

In [ ]:
plt.plot(sol.t, sol.y[0], label='s')
plt.plot(sol.t, sol.y[1], label='e')
plt.plot(sol.t, sol.y[2], label='c')
plt.plot(sol.t, sol.y[3], label='p')

#plt.ylim(0.,12.)
plt.xlim(0.,sol.t[-1])
plt.xlabel('t [h]', fontsize = 14)
plt.ylabel('concentrations [mM]', fontsize = 14)
plt.title('Enzyme reaction kinetics')
plt.legend()
plt.show()

Then we can check whether the hypotheses of the Michaelis-Menten (MM) approximation are satisfied in this case or not, and compute $p$ starting from the MM formula:
$$
\frac{dp}{dt} = k_3 \frac{e_\text{tot}s}{\frac{k_2}{k_1}+s} \,.
$$

In [ ]:
def mm_product(k1, k2, k3, e_tot, s):
    return k3*e_tot*s/(k2/k1+s)

In [ ]:
p_mm = []   # We will build here the numerical solution for the mm

# Set the initial conditions:
p_mm.append(0.)
t_prev = 0.
for t, substrate in zip(sol.t, sol.y[0]):
    p_prev = p_mm[-1]
    delta = t - t_prev
    p_next = p_prev + mm_product(k1, k2, k3, e_tot, substrate)*delta
    t_prev = t
    if delta >0:
        #if p_next > 1:
        #    p_next = 1
        p_mm.append(p_next)
        

In [ ]:
plt.plot(sol.t, sol.y[0], label='s')
plt.plot(sol.t, sol.y[1], label='e')
plt.plot(sol.t, sol.y[2], label='c')
plt.plot(sol.t, sol.y[3], label='p')
plt.plot(sol.t[10:], (sol.y[0][10:]*sol.y[1][10:]/sol.y[2][10:]), ':', label='es/c')

plt.plot(sol.t, p_mm,'b--',label='MM')

plt.ylim(0.1,10)
plt.semilogy()
plt.xlim(0.,sol.t[-1])
plt.xlabel('t [h]', fontsize = 14)
plt.ylabel('concentrations [mM]', fontsize = 14)
plt.title('Enzyme reaction kinetics')
plt.legend()
plt.show()